📚 references:

1. https://stackoverflow.com/q/19784868
2. https://stackoverflow.com/q/12129948
3. https://stackoverflow.com/q/28427236
4. https://stackoverflow.com/q/15896588

💡 [`csr_matrix` information on indexing](https://stackoverflow.com/a/52299730) \
💡 [`scipy.sparse.coo_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy-sparse-coo-matrix) \
💡 [`scipy.sparse.csr_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy-sparse-csr-matrix)
[`%%timeit` magic documentation](https://ipython.readthedocs.io/en/stable/interactive/magics.html)

### 0. imports

In [96]:
# data science
import scipy as sp
import numpy as np
# random numbers https://numpy.org/doc/stable/reference/random/index.html#random-quick-start
from numpy.random import default_rng
rng = default_rng()
# type hints
from scipy.sparse import (
    coo_matrix,
    csr_matrix,
    csc_matrix
)

### 1.1. generate sparse matrices

In [49]:
M_coo = sp.sparse.random(
    m = 20000,
    n = 20000,
    density = 0.01,
    format = 'coo',
    dtype = float,
    random_state = 42
)

In [51]:
M_csr = sp.sparse.random(
    m = 20000,
    n = 20000,
    density = 0.01,
    format = 'csr',
    dtype = float,
    random_state = 42
)

In [52]:
M_csc = sp.sparse.random(
    m = 20000,
    n = 20000,
    density = 0.01,
    format = 'csc',
    dtype = float,
    random_state = 42
)

### 1.2. generate index for matrix

In [60]:
index_matrix: np.array = np.arange(
    start = 0,
    stop = 20000,
    step = 1,
    dtype=int
)

### 1.3. generate list of rows to set to zero

In [83]:
# contains M.shape[0] 0s and 1s of a specified ratio
# 1s will be masked and then removed below, 0s will remain untouched
array_for_masking: np.array = rng.choice(
    a = [0, 1],
    size = (M.shape[0], ),
    p = [0.2, 0.8]
)

# contains only indices of rows to be set to zero 
index_matrix_zero: np.array = np.ma.masked_array(
    data = index_matrix,
    mask = array_rows
).compressed()

In [84]:
index_matrix_zero.shape

(3915,)

### 2.1. setting rows to zero
#### 2.1.1 ["multiply with a diagonal matrix"](https://stackoverflow.com/a/65364784)

In [124]:
array_for_diagonal: np.array = np.ones(
    shape = [M_coo.shape[0]],
    dtype = int
)
array_for_diagonal[index_matrix_zero] = 0

In [125]:
D = sp.sparse.diags(array_for_diagonal)

In [126]:
D

<20000x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 20000 stored elements (1 diagonals) in DIAgonal format>

In [127]:
%%timeit
M_with_some_rows_zero = M_coo @ D

179 ms ± 2.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### 2.1.2 ["take advantage of CSR format"](https://stackoverflow.com/q/12129948)

💡 [`csr_matrix` information on indexing](https://stackoverflow.com/a/52299730)

In [108]:
def csr_set_rows_to_zero(
    matrix: csr_matrix,
    rows: np.array
) -> csr_matrix:

    for row in rows:
        matrix.data[
            matrix.indptr[row]:matrix.indptr[row+1]
        ] = 0
        
    return matrix

In [109]:
%%timeit
M_csr_with_some_rows_zero= csr_set_rows_to_zero(
    matrix = M_csr,
    rows = index_matrix_zero
)